In [2]:
import pandas as pd
import numpy as np

DATA_PATH = 'C:/Users/johnp/OneDrive/Documents/development/nba_elo/data/'

## Load Data

In [50]:
tm_elo = pd.read_csv(DATA_PATH + 'nba_elo_ratings.csv')
schedule = pd.read_csv(DATA_PATH + 'nba_schedule.csv')
tm_elo

,Team,Elo Rating,Point Diff,pts_scored_poss,pts_allowed_poss
0,Atlanta Hawks,1563,0.1,116.3,116.3
1,Boston Celtics,1714,6.9,118.3,111.5
2,Brooklyn Nets,1530,1.0,115.4,114.5
3,Charlotte Hornets,1423,-6.2,109.4,115.6
4,Chicago Bulls,1548,1.3,113.8,112.5
5,Cleveland Cavaliers,1622,5.8,116.7,111.0
6,Dallas Mavericks,1586,0.3,117.2,116.9
7,Denver Nuggets,1722,3.8,118.3,114.6
8,Detroit Pistons,1291,-8.6,110.6,119.2
9,Golden State Warriors,1679,2.4,116.6,114.2


In [37]:
schedule = schedule.merge(tm_elo, left_on='Home/Neutral', right_on='Team', how='left', suffixes=('', '_drop'))
schedule.rename(columns={'Elo Rating':'Home_elo'}, inplace=True)
schedule = schedule.merge(tm_elo, left_on='Visitor/Neutral', right_on='Team', how='left', suffixes=('', '_drop'))
schedule.rename(columns={'Elo Rating':'Away_elo'}, inplace=True)

drop_cols = [col for col in schedule.columns if '_drop' in col]
schedule.drop(drop_cols, axis=1, inplace=True)
schedule_df = schedule[['Date', 'Home/Neutral', 'Visitor/Neutral', 'Home_elo', 'Away_elo']].copy()
schedule_df.rename(columns={'Home/Neutral':'Home', 'Visitor/Neutral':'Away'}, inplace=True)
schedule_df

,Date,Home,Away,Home_elo,Away_elo
0,Tue Oct 24 2023,Denver Nuggets,Los Angeles Lakers,1722,1646
1,Tue Oct 24 2023,Golden State Warriors,Phoenix Suns,1679,1616
2,Wed Oct 25 2023,Orlando Magic,Houston Rockets,1438,1319
3,Wed Oct 25 2023,New York Knicks,Boston Celtics,1610,1714
4,Wed Oct 25 2023,Indiana Pacers,Washington Wizards,1462,1509
...,...,...,...,...,...
1195,Sun Apr 14 2024,Minnesota Timberwolves,Phoenix Suns,1621,1616
1196,Sun Apr 14 2024,New Orleans Pelicans,Los Angeles Lakers,1576,1646
1197,Sun Apr 14 2024,Oklahoma City Thunder,Dallas Mavericks,1462,1586
1198,Sun Apr 14 2024,Sacramento Kings,Portland Trail Blazers,1528,1515


In [38]:
HC_BOOST = 90
schedule_df['h_adj_elo'] = schedule_df['Home_elo'] + HC_BOOST

In [39]:
def calc_win_pct(home_elo, away_elo):
    return 1 / (10 ** (-(home_elo - away_elo) / 400) + 1)

def get_winner(home_win_pct, home_tm, away_tm):
    return home_tm if np.random.binomial(1, home_win_pct) else away_tm

schedule_df['home_win_pct'] = schedule_df.apply(lambda x: calc_win_pct(x['h_adj_elo'], x['Away_elo']), axis=1)
assert schedule_df['home_win_pct'].min() > 0, 'Home win pct less than 0'
assert schedule_df['home_win_pct'].max() < 1, 'Home win pct greater than 1'
assert schedule_df['home_win_pct'].count() == len(schedule_df), 'Home win pct not calculated for all games'

## Simulate Games

In [48]:

schedule_df['home_win'] = np.random.binomial(1, schedule_df['home_win_pct'])

def sim_season(schedule_df):
    schedule_df['winner'] = schedule_df.apply(lambda x: get_winner(x['home_win_pct'], x['Home'], x['Away']), axis=1)
    win_totals = schedule_df['winner'].value_counts()
    return win_totals

win_totals = sim_season(schedule_df)
win_totals

Philadelphia 76ers        58
Denver Nuggets            57
Boston Celtics            56
Milwaukee Bucks           53
Miami Heat                53
Los Angeles Lakers        52
Dallas Mavericks          50
Golden State Warriors     50
Phoenix Suns              50
Toronto Raptors           48
Minnesota Timberwolves    48
New Orleans Pelicans      47
Los Angeles Clippers      44
Atlanta Hawks             42
Washington Wizards        41
New York Knicks           41
Cleveland Cavaliers       38
Memphis Grizzlies         36
Sacramento Kings          36
Portland Trail Blazers    35
Chicago Bulls             34
Brooklyn Nets             33
Indiana Pacers            31
Charlotte Hornets         30
Utah Jazz                 29
Orlando Magic             29
Oklahoma City Thunder     25
San Antonio Spurs         19
Houston Rockets           18
Detroit Pistons           17
Name: winner, dtype: int64

In [49]:
MC_ITERATIONS = 1000
win_totals = pd.DataFrame([sim_season(schedule_df) for _ in range(MC_ITERATIONS)])
win_totals

,Golden State Warriors,Milwaukee Bucks,Philadelphia 76ers,Denver Nuggets,Boston Celtics,Miami Heat,Toronto Raptors,Los Angeles Lakers,Atlanta Hawks,Dallas Mavericks,...,Utah Jazz,Oklahoma City Thunder,Brooklyn Nets,Portland Trail Blazers,Indiana Pacers,Orlando Magic,Charlotte Hornets,San Antonio Spurs,Detroit Pistons,Houston Rockets
winner,60,54,53,51,51,51,50,49,48,47,...,35,34,34,32,32,30,25,20,18,18
winner,51,59,56,57,54,44,41,54,34,37,...,30,32,37,39,29,32,29,23,17,19
winner,55,58,50,58,54,40,53,51,38,40,...,29,30,39,35,33,27,32,20,19,20
winner,54,54,56,57,52,50,47,49,34,43,...,30,38,32,38,25,22,29,27,20,18
winner,59,51,61,58,56,41,46,44,39,46,...,35,29,35,32,27,19,26,24,17,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
winner,53,53,51,59,50,52,52,47,39,45,...,30,32,39,35,34,25,29,18,14,16
winner,51,51,56,53,54,44,46,47,46,49,...,28,32,36,35,36,27,36,22,16,11
winner,49,51,48,66,55,50,42,45,42,35,...,30,32,41,34,26,28,26,23,15,22
winner,49,57,54,57,55,44,50,45,37,42,...,29,37,39,34,27,34,29,19,21,17
